In [1]:
import os
sep_local = os.path.sep
sep_local

'\\'

In [2]:
%env TF_KERAS = 1

env: TF_KERAS=1


In [3]:
import sys
sys.path.append('..'+sep_local+'..')

In [4]:
#done in case of linux
#import os
#os.chdir("/home/azeghost/git/Generative_Models")

In [5]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)
# print('Compute dtype: %s' % policy.compute_dtype)
# print('Variable dtype: %s' % policy.variable_dtype)

In [6]:
import tensorflow as tf

In [7]:
tf.__version__

'2.1.0'

In [8]:
dataset_name = 'pokemon'

In [9]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [10]:
IMG_DIR = 'C:\\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [11]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [12]:
inputs_shape= IMG_SIZE=(60, 60, 3)
BATCH_SIZE = 32
latent_dim = 100
intermediate_dim = 100


In [13]:
TRAIN_BUF = 600
TEST_BUF = 100


In [14]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [15]:
import tensorflow as tf

In [16]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)


In [17]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [18]:
_instance_scale

1.0

In [19]:
enc_lays2 = [
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim, dtype='float32')
]

dec_lays2 = [
    tf.keras.layers.Dense(units=16*16*32, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(16, 16, 32)),
    tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=(1, 1), padding="SAME", dtype='float32')
]

In [20]:
import numpy as np
from collections.abc import Iterable

In [21]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [22]:
_outputs_shape

10800

In [23]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

dec_lays = [tf.keras.layers.Dense(units=latent_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

## First Time

In [24]:
#from utils.data_and_files.file_utils import make_random_string
#from time import gmtime, strftime

#model_name = 'AE_' + make_random_string(5) + strftime("%a_%d_%b_%Y_%H_%M", gmtime())
#print(model_name)

In [25]:
model_name = dataset_name+'AE'

In [26]:
recoding_dir='..'+sep_local+'..'+sep_local+'recoding'+sep_local+model_name

In [27]:
import os

In [28]:
os.path.join(recoding_dir, model_name)

'..\\..\\recoding\\pokemonAE\\pokemonAE'

In [29]:
pwd

'c:\\projects\\Generatives\\Generative_Models\\notebooks\\pokemon'

In [30]:
from training.traditional.autoencoders.autoencoder import autoencoder as AE

In [31]:
inputs_shape=IMG_SIZE

In [32]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays#enc_lays2#
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays#dec_lays2#
    }
]

In [33]:
from utils.data_and_files.file_utils import create_if_not_exist

In [34]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [35]:
create_if_not_exist(_restore)
_restore

'..\\..\\recoding\\pokemonAE\\var_save_dir'

In [36]:
#to restore trained model, set filepath=_restore

In [37]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 60, 60, 3)]       0         
_________________________________________________________________
dense_2 (Dense)              (None, 60, 60, 100)       400       
_________________________________________________________________
dense_3 (Dense)              (None, 60, 60, 100)       10100     
_________________________________________________________________
flatten_1 (Flatten)          (None, 360000)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               36000100  
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 10800)             1090800   
_________________________________________________________________
reshape_1 (Reshape)          (None, 60, 60, 3)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 60, 3)         12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 60, 3)         0

  WARNING  | None


In [38]:
#ae.compile(metrics=None)
ae.compile()

Model: "pokemonAE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 60, 60, 3)]       0         
_________________________________________________________________
inference (Model)            (None, 100)               36011000  
_________________________________________________________________
generative (Model)           (None, 60, 60, 3)         1111012   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 60, 60, 3)]       0         
Total params: 37,122,012
Trainable params: 37,121,806
Non-trainable params: 206
_________________________________________________________________
None


In [39]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [40]:
progbar = NotebookPrograssBar(leave_outer=False)

In [41]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [42]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [43]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recoding\\pokemonAE\\csv_dir\\pokemonAE.csv'

In [44]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [45]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [46]:
from data.gt_load.datasets import load
DATA_PATH = '..'+sep_local+'..'+sep_local+'data'+sep_local+'.gt_datasets'

In [47]:
import numpy as np

In [48]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=50,
    epochs=100, 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True
)

Restoring model weights from the end of the best epoch.
Epoch 00057: early stopping


In [49]:
# import os

# # # TF 1.x
# # resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# # tf.tpu.experimental.initialize_tpu_system(resolver)
# # strategy = tf.distribute.experimental.TPUStrategy(resolver)

# # TF 2.0
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_host(resolver.master())
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.experimental.TPUStrategy(resolver)

# with strategy.scope():
#     #create 
#     #compile

# #fit    